In [3]:
# Cell 1: Imports & Setup
import os
import torch
import numpy as np
import pandapower as pp
import pandapower.networks as pn
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data # PyG 的核心数据结构

# 路径配置
RAW_PATH = "../data/processed/cigre_lv_stress_test_v1.pt"
SAVE_PATH = "../data/processed/cigre_lv_graph_dataset_v1.pt"

# 加载原始 Tensor 数据
if not os.path.exists(RAW_PATH):
    raise FileNotFoundError(f"❌ Raw data not found at {RAW_PATH}. Run Notebook 01 first!")

raw_data = torch.load(RAW_PATH)
X_raw = raw_data['X'].numpy()      # [N, 3] (P_tot, Q_tot, Time)
Y_v_raw = raw_data['Y_v'].numpy()  # [N, Nodes]
Y_p_raw = raw_data['Y_p'].numpy()  # [N, Nodes]

print(f"✅ Loaded raw data. Samples: {X_raw.shape[0]}")

✅ Loaded raw data. Samples: 50016


In [7]:
# Cell 2: Topology & Mapping Logic (FIXED)

def extract_graph_and_mapping(raw_y_p_dim):
    # 1. 创建网络以获取拓扑
    net = pn.create_cigre_network_lv()
    
    # --- A. 建立节点映射 (Bus ID -> 0..43 Index) ---
    bus_ids = net.bus.index.values
    # 这是一个 lookup table: 真实 Bus ID -> 连续的 0~43 索引
    bus_mapper = {b_id: i for i, b_id in enumerate(bus_ids)}
    n_nodes = len(bus_ids)
    
    # --- B. 建立负荷映射 (Load Index -> Bus Index) ---
    # 我们现在的 Y_p 是 (N, 15)，每一列对应 net.load 表的一行
    # 我们需要知道 net.load 的第 i 行连在哪个 Bus 上
    load_bus_ids = net.load.bus.values # 真实的 Bus ID
    # 将 Load 连接的 Bus ID 转换为 0~43 的索引
    load_to_bus_idx = [bus_mapper[b_id] for b_id in load_bus_ids]
    
    print(f"Mapping Check: Found {len(load_to_bus_idx)} loads mapping to {n_nodes} buses.")
    # 确保维度匹配 (生成的 Y_p 应该和 net.load 长度一致)
    if len(load_to_bus_idx) != raw_y_p_dim:
        raise ValueError(f"Mismatch! Data has {raw_y_p_dim} loads, but topology has {len(load_to_bus_idx)}.")

    # --- C. 提取边 (Edges) ---
    src = [bus_mapper[b] for b in net.line.from_bus.values]
    dst = [bus_mapper[b] for b in net.line.to_bus.values]
    
    # 提取物理参数 (R, X)
    r_ohm = net.line.r_ohm_per_km.values * net.line.length_km.values
    x_ohm = net.line.x_ohm_per_km.values * net.line.length_km.values
    
    # 变压器边
    t_src = [bus_mapper[b] for b in net.trafo.hv_bus.values]
    t_dst = [bus_mapper[b] for b in net.trafo.lv_bus.values]
    t_r = np.array([0.001] * len(net.trafo))
    t_x = np.array([0.01] * len(net.trafo))
    
    # 构建双向图
    final_src = src + t_src + dst + t_dst
    final_dst = dst + t_dst + src + t_src
    final_r = np.concatenate([r_ohm, t_r, r_ohm, t_r])
    final_x = np.concatenate([x_ohm, t_x, x_ohm, t_x])
    
    edge_index = torch.tensor([final_src, final_dst], dtype=torch.long)
    edge_attr = torch.tensor(np.stack([final_r, final_x], axis=1), dtype=torch.float)
    
    return edge_index, edge_attr, load_to_bus_idx, n_nodes

# 执行提取
# 注意：X_raw 等变量需要来自 Cell 1 的加载
# Y_p_raw.shape[1] 应该是 15
edge_index, edge_attr, load_map, n_nodes = extract_graph_and_mapping(Y_p_raw.shape[1])

print(f"🔗 Corrected Topology:")
print(f"   Nodes: {n_nodes} (Expected 44)")
print(f"   Load Map indices: {load_map}")

Mapping Check: Found 15 loads mapping to 44 buses.
🔗 Corrected Topology:
   Nodes: 44 (Expected 44)
   Load Map indices: [2, 12, 16, 17, 18, 19, 22, 24, 35, 36, 37, 40, 41, 42, 43]


In [8]:
# Cell 3: Expand Power Injections & Normalization

def process_and_scale(X, Y_v, Y_p_sparse, load_map, n_nodes):
    """
    Y_p_sparse: [N, 15] -> 只有负荷点的功率
    load_map: [15] -> 负荷点对应的节点索引
    target: [N, 44] -> 全网节点功率 (非负荷点填 0)
    """
    n_samples = X.shape[0]
    
    # 1. 扩展功率矩阵 (Map 15 cols to 44 cols)
    print("Expanding Power Injections to Full Bus Matrix...")
    Y_p_full = np.zeros((n_samples, n_nodes), dtype=np.float32)
    
    # 将稀疏的负荷数据填入对应的节点列
    # Y_p_sparse[:, i] 是第 i 个负荷的所有样本数据
    # load_map[i] 是第 i 个负荷所在的节点索引
    for i, bus_idx in enumerate(load_map):
        Y_p_full[:, bus_idx] = Y_p_sparse[:, i]
        
    print(f"   Expansion done. Y_p shape: {Y_p_full.shape}")

    # 2. 划分数据集
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    train_idx = indices[:int(0.8*n_samples)]
    val_idx = indices[int(0.8*n_samples):int(0.9*n_samples)]
    test_idx = indices[int(0.9*n_samples):]
    
    # 3. 标准化 (Scaling)
    # 只在 Train 上 fit
    scaler_x = StandardScaler().fit(X[train_idx, :2])
    scaler_y_v = StandardScaler().fit(Y_v[train_idx])
    scaler_y_p = StandardScaler().fit(Y_p_full[train_idx]) # 现在是对 44 列做 scaling
    
    # Transform
    X_norm = X.copy()
    X_norm[:, :2] = scaler_x.transform(X[:, :2])
    Y_v_norm = scaler_y_v.transform(Y_v)
    Y_p_norm = scaler_y_p.transform(Y_p_full)
    
    # 转 Tensor
    return (torch.from_numpy(X_norm).float(),
            torch.from_numpy(Y_v_norm).float(),
            torch.from_numpy(Y_p_norm).float(),
            train_idx, val_idx, test_idx,
            {'x': scaler_x, 'y_v': scaler_y_v, 'y_p': scaler_y_p})

# 执行处理
X_norm, Y_v_norm, Y_p_norm, train_idx, val_idx, test_idx, scalers = \
    process_and_scale(X_raw, Y_v_raw, Y_p_raw, load_map, n_nodes)

print(f"📊 Final Data Shapes:")
print(f"   Y_v (Voltage): {Y_v_norm.shape} (Should be N, 44)")
print(f"   Y_p (Power):   {Y_p_norm.shape} (Should be N, 44)")

Expanding Power Injections to Full Bus Matrix...
   Expansion done. Y_p shape: (50016, 44)
📊 Final Data Shapes:
   Y_v (Voltage): torch.Size([50016, 44]) (Should be N, 44)
   Y_p (Power):   torch.Size([50016, 44]) (Should be N, 44)


In [9]:
# Cell 4: Save
processed_data = {
    "edge_index": edge_index,
    "edge_attr": edge_attr,
    "X": X_norm,
    "Y_v": Y_v_norm,
    "Y_p": Y_p_norm,
    "train_idx": torch.from_numpy(train_idx),
    "val_idx": torch.from_numpy(val_idx),
    "test_idx": torch.from_numpy(test_idx),
    "scalers": scalers,
    "n_nodes": n_nodes
}
torch.save(processed_data, SAVE_PATH)
print("✅ Saved corrected dataset.")

✅ Saved corrected dataset.
